In [2]:
import pyodbc
import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

# defining the server and the database
server = '(localdb)\mssqllocaldb' 
database = 'AdventureWorksDW2012'  

# Define the connection string
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server}; \
    SERVER='+ server +'; \
    DATABASE ='+ database +';\
    Trusted_Connection=yes;'
)

### 1. How many product categories does AdventureWorks carry?
#### Thought Process:
#### * Used database: [AdventureWorksDW2012].[dbo].[DimProductCategory] AS DimProduct
#### * Used SELECT DISTINCT statement to pull unique Product Category Key only once
#### * Then included a COUNT() function to get the total number of unique keys

In [3]:
data = pd.read_sql_query("""SELECT DISTINCT COUNT(Cat.ProductCategoryKey) AS [Product Categories]
                            FROM [AdventureWorksDW2012].[dbo].[DimProductCategory] AS Cat""",conn)
data

,Product Categories
0,4


### 2. Get the list (in English) of products which are colored Black or Silver.
### Make sure the product status is current.
#### Thought Process:
#### * Used database: [AdventureWorksDW2012].[dbo].[DimProduct] AS DimProduct
#### * Using Status and Color field in the WHERE clause to filter records in table
#### * Color must be Black and SIlver and Status must be current

In [68]:

data = pd.read_sql_query("""SELECT DimProduct.EnglishProductName AS [Product List],
                            Color AS [Product Color]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS DimProduct
                            WHERE DimProduct.Status = 'Current'
                            AND DimProduct.Color in ('Black', 'Silver')
                            ORDER BY DimProduct.Color""",conn)
data

,Product List,Product Color
0,LL Crankarm,Black
1,ML Crankarm,Black
2,HL Crankarm,Black
3,Chainring,Black
4,"HL Road Frame - Black, 58",Black
...,...,...
81,Front Derailleur Linkage,Silver
82,Lock Ring,Silver
83,Rear Derailleur Cage,Silver
84,Chainring Bolts,Silver


### 3. Get the list (in English) of products which have no known color but they
### have the letters “caps” in their name. Make sure the product status is current.
#### Thought Process:
#### * Used database [AdventureWorksDW2012].[dbo].[DimProduct] AS DimProduct
#### * Used LIKE operator to match records that contain caps in Product Name value
#### * Used AND operators to combine two more conditions
#### * Status must be current and Color must be NA

In [34]:
data = pd.read_sql_query("""SELECT DimProduct.EnglishProductName AS [Product List],
                            Color AS [Product Color]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS DimProduct
                            WHERE DimProduct.Color = 'NA' AND DimProduct.status = 'Current'
                            AND DimProduct.EnglishProductName LIKE '%Caps%'""",conn)
data

,Product List,Product Color
0,Mountain End Caps,NA
1,Road End Caps,NA
2,Touring End Caps,NA


### 4. Show the unit price (call it Price), discount, and product standard cost on
### all internet sales records which were shipped on '2005-07-10'
#### Thought Process:
#### * Used database [AdventureWorksDW2012].[dbo].[FactInternetSales] AS InternetSales
#### * Pulled records with the ship date 2005-07-10 in InternetSales table
#### * Used FORMAT(column,'C') to change format from decimal to currency ($)

In [64]:
data = pd.read_sql_query("""SELECT FORMAT(InternetSales.UnitPrice,'C') AS Price, 
                            FORMAT(InternetSales.DiscountAmount,'C') AS [Discount Amount],
                            FORMAT(InternetSales.ProductStandardCost,'C') AS [Standard Product Cost]
                            FROM [AdventureWorksDW2012].[dbo].[FactInternetSales] AS InternetSales
                            WHERE InternetSales.ShipDate = '2005-07-10'""",conn)
data

,Price,Discount Amount,Standard Product Cost
0,"$3,578.27",$0.00,"$2,171.29"
1,"$3,578.27",$0.00,"$2,171.29"
2,$699.10,$0.00,$413.15
3,"$3,578.27",$0.00,"$2,171.29"
4,"$3,578.27",$0.00,"$2,171.29"


### 5. List the last name, first name, and gender (coma separated in one field),
### birthday and marital status of all customers who have some college
### education, are male, yearly income is less then or equal 60K, have 2 or
### more kids, and their first name starts with ‘Jo…’ (must order by birthdate)
#### Thought Process:
#### Used + ', ' + to concatenate LastName, FirstName and Gender fields
#### Used AND operators to combine multiple conditions
#### Used ORDER BY keyword to sort Birth Date in ascending order


In [57]:
data = pd.read_sql_query("""SELECT Customer.LastName + ', ' + Customer.FirstName + ', ' + Customer.Gender AS [Customer Name],
                            Customer.BirthDate AS [Birth Date], Customer.MaritalStatus AS [Marital Status], 
                            FORMAT(Customer.YearlyIncome,'C') AS [Yearly Income],
                            Customer.TotalChildren AS [Total Children]
                            FROM [AdventureWorksDW2012].[dbo].[DimCustomer] AS Customer
                            WHERE Customer.EnglishEducation = 'Partial College' 
                            AND Customer.Gender IN ('M')
                            AND Customer.YearlyIncome <= 60000 
                            AND Customer.TotalChildren  >= 2
                            AND Customer.FirstName LIKE 'Jo%'
                            ORDER BY Customer.BirthDate""",conn)
data

,Customer Name,Birth Date,Marital Status,Yearly Income,Total Children
0,"Williams, Joshua, M",1932-12-24,M,"$40,000.00",3
1,"Shen, Jon, M",1940-05-15,M,"$10,000.00",2
2,"Luo, Johnny, M",1946-05-27,M,"$30,000.00",2
3,"Butler, Jose, M",1946-12-10,S,"$30,000.00",2
4,"Robinson, Jonathan, M",1948-09-22,M,"$60,000.00",2
5,"Kumar, Jose, M",1949-03-09,M,"$60,000.00",2
6,"Diaz, Jose, M",1949-05-09,M,"$60,000.00",2
7,"Phillips, Jose, M",1949-06-09,M,"$60,000.00",2
8,"Clark, John, M",1949-07-23,M,"$60,000.00",2
9,"Lewis, Joseph, M",1950-04-12,M,"$60,000.00",2
